In [425]:
import os
import pandas as pd
import json


pd.set_option('display.float_format', '{:.2f}'.format)


In [426]:


# Define the folder number
folder_number = '13'  # Change this to the specific folder number you are working with

# Define the paths to the JSON files
base_path = f'results_Aug1/slow_liquidity/try 2/'
channels_json_path = os.path.join(base_path, 'channels.json')
forwarding_history_json_path = os.path.join(base_path, 'forwarding_history.json')




In [427]:


# Load channels.json
with open(channels_json_path, 'r') as file:
    channels_data = json.load(file)

# Load forwarding_history.json
with open(forwarding_history_json_path, 'r') as file:
    forwarding_history_data = json.load(file)

# Extract the forwarding events
forwards = forwarding_history_data.get('forwards', [])

# Create the initial DataFrame
rows = []
for entry in forwards:
    if isinstance(entry, dict):
        rows.append({
            'addTimeNs': int(entry['addTimeNs']),
            'resolveTimeNs': int(entry.get('resolveTimeNs', 0)),
            'eventTimeNs': int(entry['addTimeNs']),
            'eventType': 'add',
            'incomingAmount': int(entry['incomingAmount']),
            'outgoingAmount': int(entry['outgoingAmount']),
            'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
            'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
        })
        if 'resolveTimeNs' in entry:
            rows.append({
                'addTimeNs': int(entry['addTimeNs']),
                'resolveTimeNs': int(entry['resolveTimeNs']),
                'eventTimeNs': int(entry['resolveTimeNs']),
                'eventType': 'resolve',
                'incomingAmount': int(entry['incomingAmount']),
                'outgoingAmount': int(entry['outgoingAmount']),
                'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
            })

pair_schedule_df = pd.DataFrame(rows)

# Sort the DataFrame by eventTimeNs
pair_schedule_df = pair_schedule_df.sort_values(by='eventTimeNs').reset_index(drop=True)

In [428]:
# Create a DataFrame for channel capacities
channels_df = pd.DataFrame(channels_data['channels'])
channels_df['chan_id'] = channels_df['chan_id'].astype(int)
channels_df['capacity'] = channels_df['capacity'].astype(int)*1000
channels_df = channels_df[['chan_id', 'capacity']]

In [429]:
def track_funds(pair_schedule_df, channels_df, jam_lim = 0.2):
    # Initialize a dictionary to track available funds
    #funds = {channel: {'incoming': cap // 2, 'outgoing': cap // 2} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}
    funds = {channel: {'incoming': 0, 'outgoing': 0} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}

    
    # List to store times when locked funds exceed 80%
    high_locked_times = []
    status = []
    
    for _, row in pair_schedule_df.iterrows():
        incoming_id = row['shortChannelId_incoming']
        outgoing_id = row['shortChannelId_outgoing']

        # Update funds based on eventType
        if row['eventType'] == 'add':
            if incoming_id in funds:
                funds[incoming_id]['outgoing'] += row['incomingAmount']
            if outgoing_id in funds:
                funds[outgoing_id]['incoming'] += row['outgoingAmount']
        elif row['eventType'] == 'resolve':
            if incoming_id in funds:
                funds[incoming_id]['outgoing'] -= row['incomingAmount']
            if outgoing_id in funds:
                funds[outgoing_id]['incoming'] -= row['outgoingAmount']

        # Check if the locked amounts exceed 80% of available funds
        for channel, cap in zip(channels_df['chan_id'], channels_df['capacity']):
            if channel in funds:
                status.append({
                    'time': row['eventTimeNs'],
                    'channel': channel,
                    'capacity': cap,
                    'incoming_locked': funds[channel]['incoming'],
                    'outgoing_locked': funds[channel]['outgoing']
                })
                if funds[channel]['incoming'] > jam_lim * (cap // 2):
                    high_locked_times.append({
                        'time': row['eventTimeNs'],
                        'resolve_time': row['resolveTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'direction': 'incoming',
                        'funds_locked': funds[channel]['incoming'],
                    })
                if funds[channel]['outgoing'] > jam_lim * (cap // 2):
                    high_locked_times.append({
                        'time': row['eventTimeNs'],
                        'resolve_time': row['resolveTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'direction': 'outgoing',
                        'funds_locked': funds[channel]['outgoing']
                    })
    return pd.DataFrame(high_locked_times), pd.DataFrame(status)

liq_jam_ratio = 0.9
high_locked_df, st_df = track_funds(pair_schedule_df, channels_df, liq_jam_ratio)


In [430]:
pair_schedule_df.head(10)

,addTimeNs,resolveTimeNs,eventTimeNs,eventType,incomingAmount,outgoingAmount,shortChannelId_outgoing,shortChannelId_incoming
0,1721924901874273832,1721924907080224832,1721924901874273832,add,1053,53,393625162809344,378232000020480
1,1721924901874273832,1721924907080224832,1721924907080224832,resolve,1053,53,393625162809344,378232000020480
2,1721924926575783832,1721924928923879832,1721924926575783832,add,1053,53,393625162809344,378232000020480
3,1721924926575783832,1721924928923879832,1721924928923879832,resolve,1053,53,393625162809344,378232000020480
4,1721924956519197832,1721924959418218832,1721924956519197832,add,1053,53,393625162809344,378232000020480
5,1721924956519197832,1721924959418218832,1721924959418218832,resolve,1053,53,393625162809344,378232000020480
6,1721924980532311832,1721924988626147832,1721924980532311832,add,1053,53,393625162809344,378232000020480
7,1721924981693596832,1721924988628274832,1721924981693596832,add,1002,2,393625162809344,378232000020480
8,1721924980532311832,1721924988626147832,1721924988626147832,resolve,1053,53,393625162809344,378232000020480
9,1721924981693596832,1721924988628274832,1721924988628274832,resolve,1002,2,393625162809344,378232000020480


In [431]:
st_df.head()

,time,channel,capacity,incoming_locked,outgoing_locked
0,1721924901874273832,379331511648256,2000000000,0,0
1,1721924901874273832,404620279087104,25000000000,0,0
2,1721924901874273832,393625162809344,100000000000,53,0
3,1721924901874273832,361739325603840,3000000000,0,0
4,1721924901874273832,367236883742720,84803717000,0,0


In [432]:
high_locked_df.head(20)

,time,resolve_time,channel,capacity,direction,funds_locked
0,1722533527875928510,1722533527975803513,393625162809344,100000000000,incoming,45082259000
1,1722533527904326585,1722533528000942907,393625162809344,100000000000,incoming,45082259000
2,1722533527918785597,1722533528119242439,393625162809344,100000000000,incoming,50082265000
3,1722533527929374760,1722533528143247197,393625162809344,100000000000,incoming,55082271000
4,1722533527943018417,1722533527943018417,393625162809344,100000000000,incoming,50082265000
5,1722533527958919091,1722533527958919091,393625162809344,100000000000,incoming,45082259000
6,1722533528083901508,1722533528232548143,393625162809344,100000000000,incoming,45082259000
7,1722533528104308912,1722533528248124849,393625162809344,100000000000,incoming,50082265000
8,1722533528119242439,1722533528119242439,393625162809344,100000000000,incoming,45082259000
9,1722533528171711041,1722533528273548724,393625162809344,100000000000,incoming,45082259000


In [433]:
high_locked_df[high_locked_df['channel']==379331511648256]

,time,resolve_time,channel,capacity,direction,funds_locked


# count jammed time

In [434]:
def calculate_active_time(df):
    # Sort by start time
    df = df.sort_values(by='time')
    
    # Initialize variables
    total_active_time = 0
    current_start = None
    current_end = None
    
    for _, row in df.iterrows():
        start, end = row['time'], row['resolve_time']
        
        if current_start is None:
            current_start = start
            current_end = end
        else:
            if start <= current_end:
                # Overlapping period, extend the end time if needed
                current_end = max(current_end, end)
            else:
                # Non-overlapping period, add the previous period's duration
                total_active_time += current_end - current_start
                current_start = start
                current_end = end
    
    # Add the last period
    if current_start is not None:
        total_active_time += current_end - current_start
    
    return total_active_time

grouped = high_locked_df.groupby(['channel', 'direction'])

results = grouped.apply(calculate_active_time).reset_index()
results.columns = ['channel', 'direction', 'total_active_time']

# Step 5: Print the results
print(results)

           channel direction  total_active_time
0  393625162809344  incoming       670178244285


In [435]:
results['total_active_time_minutes'] = results['total_active_time']/60000000000

In [436]:
results

,channel,direction,total_active_time,total_active_time_minutes
0,393625162809344,incoming,670178244285,11.17
